In [3]:
#4KAAC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from decimal import Decimal
import pandas as pd
import time


driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20) 

bus_route_link = []
bus_route_name = []
bus_route_final = []
#bus_route = []
bus_names = [] 
bus_type = [] 
departing_time = []
duration = [] 
Reaching_time = [] 
star_rating = [] 
price = [] 
seat_availablity = []

def jump_pages():
    for page_number in range(1,3):
        find_bus_routes = driver.find_elements(By.CLASS_NAME,'route')
        if page_number < 3:
            for route in find_bus_routes:
                route_name = route.get_attribute("title")
                route_link = route.get_attribute("href")
                bus_route_name.append(route_name)
                bus_route_link.append(route_link)
                print(route_name,route_link)
            try:
                pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "DC_117_paginationTable")]')))
                next_page_button=pagination_container.find_element(By.XPATH, f'//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]')
                action = ActionChains(driver)
                action.move_to_element(next_page_button).perform()
                time.sleep(2)
                next_page_button.click()
            except:
                print('failure')

def scroll(): # function to scroll page from top to bottom
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")# Scroll down to the bottom in order to load all the buses
        time.sleep(2)# Wait for page to load
        new_height = driver.execute_script("return document.body.scrollHeight")# Calculate new scroll height and compare with last scroll height
        if new_height == last_height:# If it is the same height then it is at the end of the page
            break
        last_height = new_height

def click_button(): # function to click button for view buses for government buses
    view_buses = driver.find_elements(By.CSS_SELECTOR, '.p-left-10.icon.icon-down')
    if view_buses:
        button_count = len(view_buses)
        for i in range(button_count-1 , -1, -1):
            view_buses[i].click()

def get_bus_details(): # function to get all bus details
    rate = driver.find_elements(By.CSS_SELECTOR,".fare.d-block")#
    seat_available = driver.find_elements(By.CSS_SELECTOR,".column-eight.w-15.fl")
    rating = driver.find_elements(By.CSS_SELECTOR,".column-six.p-right-10.w-10.fl")
    names = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
    b_type = driver.find_elements(By.CSS_SELECTOR,".bus-type.f-12.m-top-16.l-color.evBus")
    start_time = driver.find_elements(By.CSS_SELECTOR,".dp-time.f-19.d-color.f-bold")
    total_time = driver.find_elements(By.CSS_SELECTOR,".dur.l-color.lh-24")
    end_time =  driver.find_elements(By.CSS_SELECTOR,".bp-time.f-19.d-color.disp-Inline")
    
    time.sleep(5)
    print(len(names),len(b_type),len(start_time),len(total_time),len(end_time),len(rating),len(rate),len(seat_available))

    for i in range(0,len(seat_available)):#(len(rating)):
        bus_names.append(names[i].text)
        bus_type.append(b_type[i].text)
        departing_time.append(start_time[i].text)
        duration.append(total_time[i].text)
        Reaching_time.append(end_time[i].text)
        if (rating[i].text) == "New":
            star_rating.append(0.0)
        else:
            star_rating.append((rating[i].text)[0:3])
        price.append((rate[i].text))
        seat_availablity.append((seat_available[i].text)[0:2])
    
    print(len(bus_names),len(bus_type),len(departing_time),len(duration),len(Reaching_time),len(star_rating),len(price),len(seat_availablity))


driver.get("https://www.redbus.in/online-booking/kaac-transport")
driver.maximize_window()
time.sleep(5)

jump_pages()
    
print("Total routes",len(bus_route_name))

for i in range (0,len(bus_route_link)):# used 2 just to check if every thing is working as expected
    driver.get(bus_route_link[i])
    body = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.clearfix.bus-item')))
    click_button()
    scroll()
    time.sleep(5)
    get_bus_details()
    temp_name = []
    temp_link = []
    bus_route = []
    state_service = []
    print(len(bus_names))
    for j in range(len(bus_names)):
        temp_name.append(bus_route_name[i])
        temp_link.append(bus_route_link[i])
        state_service.append("KAAC")
    print(len(temp_name),len(temp_link))
    bus_route = list(zip(state_service,temp_name,temp_link,bus_names,bus_type,departing_time,duration,Reaching_time,star_rating,price,seat_availablity))
    bus_route_final.extend(bus_route)
    print(i+1,bus_route_name[i],"completed")
    temp_name.clear()
    temp_link.clear()
    state_service.clear()
    bus_route.clear()
    bus_names.clear()
    bus_type.clear()
    departing_time.clear()
    duration.clear()
    Reaching_time.clear()
    star_rating.clear()
    price.clear()
    seat_availablity.clear()
    
df_bus_details = pd.DataFrame(bus_route_final, columns=["Service Name","Route_name", "Route_link", "Bus_names","Bus_type","Departing_time","Duration","Reaching_time","Star_rating","Price","Seat_availablity"])
df_bus_details['Seat_availablity']=df_bus_details['Seat_availablity'].astype(int)
df_bus_details['Price'] = df_bus_details['Price'].str.replace('INR ', '', regex=False)
df_bus_details['Star_rating'] = df_bus_details['Star_rating'].str.replace(' ', '', regex=False)
df_bus_details['Price'] = df_bus_details['Price'].astype(float)
df_bus_details['Star_rating'] = df_bus_details['Star_rating'].astype(float)

Diphu to Guwahati https://www.redbus.in/bus-tickets/diphu-to-guwahati
Bokolia (assam) to Guwahati https://www.redbus.in/bus-tickets/bokolia-assam-to-guwahati
Guwahati to Diphu https://www.redbus.in/bus-tickets/guwahati-to-diphu
Dokmoka to Guwahati https://www.redbus.in/bus-tickets/dokmoka-to-guwahati
Langhin (assam) to Guwahati https://www.redbus.in/bus-tickets/langhin-assam-to-guwahati
Guwahati to Dokmoka https://www.redbus.in/bus-tickets/guwahati-to-dokmoka
Guwahati to Langhin (assam) https://www.redbus.in/bus-tickets/guwahati-to-langhin-assam
Guwahati to Bokolia (assam) https://www.redbus.in/bus-tickets/guwahati-to-bokolia-assam
Diphu to Hamren https://www.redbus.in/bus-tickets/diphu-to-hamren
Manja (assam) to Guwahati https://www.redbus.in/bus-tickets/manja-assam-to-guwahati
Guwahati to Manja (assam) https://www.redbus.in/bus-tickets/guwahati-to-manja-assam
Diphu to Jagiroad https://www.redbus.in/bus-tickets/diphu-to-jagiroad
Guwahati to Hamren https://www.redbus.in/bus-tickets/guw

In [5]:
df_bus_details

,Service Name,Route_name,Route_link,Bus_names,Bus_type,Departing_time,Duration,Reaching_time,Star_rating,Price,Seat_availablity
0,KAAC,Diphu to Guwahati,https://www.redbus.in/bus-tickets/diphu-to-guw...,KAAC TRANSPORT - 157249,Non AC Seater 2+2,04:00,05h 30m,09:30,4.1,361.0,8
1,KAAC,Diphu to Guwahati,https://www.redbus.in/bus-tickets/diphu-to-guw...,Kareng Travels,A/C Seater (2+1),04:45,05h 15m,10:00,3.6,400.0,13
2,KAAC,Diphu to Guwahati,https://www.redbus.in/bus-tickets/diphu-to-guw...,Kareng Travels,Non AC Seater (2+1),11:15,05h 30m,16:45,4.3,380.0,19
3,KAAC,Bokolia (assam) to Guwahati,https://www.redbus.in/bus-tickets/bokolia-assa...,KAAC TRANSPORT - 157249,Non AC Seater 2+2,05:00,04h 30m,09:30,3.7,299.0,11
4,KAAC,Bokolia (assam) to Guwahati,https://www.redbus.in/bus-tickets/bokolia-assa...,Kareng Travels,Non AC Seater (2+1),12:15,04h 30m,16:45,4.1,300.0,27
5,KAAC,Bokolia (assam) to Guwahati,https://www.redbus.in/bus-tickets/bokolia-assa...,Kareng Travels,A/C Seater (2+1),06:00,04h 00m,10:00,4.0,300.0,15
6,KAAC,Guwahati to Diphu,https://www.redbus.in/bus-tickets/guwahati-to-...,KAAC TRANSPORT - 157250,Non AC Seater 2+2,11:30,06h 15m,17:45,4.5,361.0,40
7,KAAC,Guwahati to Diphu,https://www.redbus.in/bus-tickets/guwahati-to-...,Kareng Travels,Non AC Seater (2+1),05:00,06h 00m,11:00,4.1,380.0,11
8,KAAC,Guwahati to Diphu,https://www.redbus.in/bus-tickets/guwahati-to-...,Kareng Travels,A/C Seater (2+1),09:30,06h 45m,16:15,4.5,400.0,26
9,KAAC,Guwahati to Diphu,https://www.redbus.in/bus-tickets/guwahati-to-...,Kareng Travels,NON A/C Seater Push Back (2+2),11:45,06h 45m,18:30,4.4,361.0,22


In [1]:
df_bus_details['Star_rating'] = df_bus_details['Star_rating'].astype(float)
df_bus_details.info()

NameError: name 'df_bus_details' is not defined

In [11]:
df_bus_details.to_csv("KAAC.csv",index = False)